This file controls the neuroevolution agent in a "decisive" way, i.e., by only feeding the agent's neural network with the actionable states of the board to minimize the need to run the neural network and therefore computational effort.

This code is by no means perfect, but it does produce agents with reasonably serviceable play.

In [14]:
# play the game directly ('X' to roll gems; arrow keys to move, Esc to close)
# !python -m retro.examples.interactive --game Columns-Genesis

In [1]:
## %matplotlib auto
# import relevant libraries
import retro 
import numpy as np 
import cv2
import neat
import pickle
import time
import random
import warnings
from scipy.ndimage.measurements import label as sp_label
# ignore warnings
warnings.filterwarnings('ignore')
#set a random seed
random.seed(12345)

In [2]:
def rescale_grays():
    '''
    Remap grayscale image to new, higher contrast values
    
        input: (N/A)

       output: rescale_map (NumPy array): color mapping for use in preprocessing
    
    called by: eval_genomes
    '''    
    
    ######## uncomment block below to test standalone function ########
#     game, state = 'Columns-Genesis', 'Arcade.Easy.Level0'
#     # specify type of retro environment observation space via index of below list
#     input_type_index = 0 
#     # specify observation type
#     input_types = [retro.Observations.IMAGE, retro.Observations.RAM]
#     # create the retro environment for the chosen game, including starting state
#     env = retro.make(game,state,obs_type=input_types[input_type_index],use_restricted_actions=retro.Actions.ALL            )
#     ob = env.reset()
    ######## uncomment block above to test standalone function ########
    ob = env.reset()
    # color mapping start and end states to improve contrast and homogenize colors
    colors =     np.array([192,156,138, 62,166, 90, 92, 46,128,121,69,59])
    new_colors = np.array([202,202,172,172,142,142,112,112, 82, 82,52,52])
    # initialize the mapping array
    reshade_map = np.zeros(256, dtype=ob.dtype) 
    # populate the display mapping array
    reshade_map[colors] = new_colors
    # add 1 unit of 'wiggle room'
    reshade_map = reshade_map + np.roll(reshade_map, 1) 
#     reshade_map[0]=0; reshade_map[-1]=0
    # return final color mapping
#     print(' ran rescale_grays to make map with type', type(reshade_map))
    return reshade_map

######## uncomment below line to test standalone function ########
# rescale_grays()

# index guide for mapping array:
#   0              3              6              9              12 

In [3]:
def process_image(ob_img, map, dims=np.array([8, 208, 16, 96]), shrink=8):
    '''
     input: ob_img (NumPy array): image to be processed
            map(NumPy array): remapping of grayscale to improve contrast for the 
            neural net input
            dims (NumPy array): dimensions of cropped observation frame
            shrink (int): shrink factor by which to rescale image
    output: prepped_image (NumPy array):
    
    called by: eval_genomes
    '''
# prepare a scaled image for rendering of final observation
    sc_dims = dims/shrink
    
    # apply color conversion (quirk of OpenCV)
    img  = cv2.cvtColor(ob_img, cv2.COLOR_BGR2GRAY)
#             if frame == 1:
#                 print(f'initial input_img: {input_img.shape}')
    # crop the image via NumPy slicing (dims indices defined above)
    img = img[dims[0]:dims[0]+dims[1], dims[2]:dims[2]+dims[3]]
#             if frame == 1:
#                 print(f'cropped input_img: {input_img.shape}')
    # rescale to minimum dimensions that accommodate block cycling animation
    img  = cv2.resize(img, (int((sc_dims[3])/2), int(sc_dims[1])))
#             if frame == 1:
#                 print(f'final input_img: {input_img.shape}')
    # remap the colors to homogenize the jewel colorings
    resh_img = map[img]
    # throw out now-unnecessary vertical resolution
    prepped_image = resh_img[::2]      
#     print('ran process_image to produce: \', prepped_image)    
    return prepped_image

In [4]:
def col_isle_fit_calc(image_array, col_hts_i, isle_sizes_i):
    '''
     Calculate column height penalty and island bonus for provided array and update
     initial values for the next calculation
     
        input: image_array (NumPy array): processed image of current game state 
               col_hts_i (NumPy array): initial state of column heights 
               isle_sizes_i (NumPy array): initial state of island sizes 

       output: col_pen (float): column penalty term associated with provided game state 
               isle_bonus (float): isle bonus term associated with provided game state
               col_hts_i (NumPy array): init. state of col. heights for next iteration 
               isle_sizes_i (NumPy array): init. state of isle sizes for next iteration
            
    called by: 
    '''
    # define valid connection structure for column labeling
    c_valid_conns = np.array((0,1,0,0,1,0,0,1,0,), dtype=np.int).reshape((3,3))
    # define valid connection structure for island labeling
    i_valid_conns = np.append(np.append(np.zeros(9),np.ones(9)),np.zeros(9)).reshape(3,3,3)
    # initialize index array for color-broadcasted game state
    ind_grid = np.indices((7,13,6))     
    # match range of colors to range of label outputs
    unit_img = np.where(image_array == 0, 0, (image_array - 22)/30)
    
    ######## column penalty calculation ########
    
    # c_ncomponents is a simple count of the conected columns in labeled
    columns, c_ncomponents = sp_label(unit_img, c_valid_conns)
#     print(columns)

    # throw out the falling block with .isin(x,x[-1]) combined with... 
    # the mask nonzero(x) 
    drop_falling = np.isin(columns, columns[-1][np.nonzero(columns[-1])])
    col_hts = drop_falling.sum(axis=0)
#     print(f'col_hts {col_hts}')

    # calculate differentials for the (grounded) column heights
    d_col_hts = np.sum(col_hts - col_hts_i)
#     print(f'col_hts {col_hts} - col_hts_i {col_hts_i} ===> d_col_hts {d_col_hts}')

    # set col_hts_i to current col_hts for next evaluation
    col_hts_i = col_hts
    
    # calculate penalty/bonus function
    col_pen = np.where(d_col_hts > 0, (col_hts**4 - 3**4), 0).sum()           
#     if col_pen !=0:
#         print(f'col_pen: {col_pen}')

    ######## end column penalty calculation ########

    ######## color island bonus calculation ########
    
    # mask the unit_img to remove the falling block
    isle_img = drop_falling * image_array
#             print(isle_img)

    # broadcast the game board to add a layer for each color
    isle_imgs = np.broadcast_to(isle_img,(7,*isle_img.shape))
    
    # define a mask to discriminate on color in each layer
    isle_masked = isle_imgs*[isle_imgs==ind_grid[0]]
    
    # reshape the array to return to 3 dimensions
    isle_masked = isle_masked.reshape(isle_imgs.shape)
    
    # generate the isle labels
    isle_labels, isle_ncomps = sp_label(isle_masked, i_valid_conns)
    
    # determine the island sizes (via return_counts) for all the unique labels
    isle_inds, isle_sizes = np.unique(isle_labels, return_counts=True)
    
    # zero out isle_sizes[0] to remove spike for background (500+ for near empty board)
    isle_sizes[0] = 0
    
    # evaluate difference to determine whether bonus applies
    if isle_sizes_i.sum() != isle_sizes.sum():
        
    # calculate bonus for all island sizes ater throwing away the 0 count
        isle_bonus = (isle_sizes**3).sum()
        
    else:
        isle_bonus = 0
#             if isle_bonus != 0:
#                 print(f'isle_bonus:{isle_bonus} isle_avgs_i: {isle_sizes_i.sum()}  isle_avgs {isle_sizes.sum()}')

    # update the size distribution from the previous frame
    isle_sizes_i = isle_sizes
    ######## color island bonus calculation ########
    
#     print('col_isle_fit_calc to generate: ', col_pen, isle_bonus, col_hts_i, isle_sizes_i)
    return col_pen, isle_bonus, col_hts_i, isle_sizes_i

In [5]:
def score_future_states(image_array, col_scale, isle_scale):
    '''
     input: image_array (NumPy array): processed image of current game state
            col_scale (float): scaling factor for column penalty (should be < 0)
            isle_scale (float/int): scaling factor for isle bonus
    
    output: future_state_scores (NumPy array): evaluation values of future possible states
            evaluations are based on the scaled column penalty and island bonus 
            fitness terms 
    
    called by: eval_genomes
    '''

    # match range of colors to range of label outputs
    unit_img = np.where(image_array == 0, 0, (image_array - 22)/30)

    # isolate the falling block
    falling = unit_img[:3,3] 
#     print('original state of falling: ', falling)

    # capture permutations and transpose to retain original orientation of results
    falling_perms = np.append(np.append(falling, np.roll(falling,1)), 
                             np.roll(falling,-1)                    ).reshape((3,3)).T
#     print('falling permutations array: \n', falling_perms)

    # for this function, dropping the falling brick always happens in one place
    image_array[:3,3] = 0  
    
    drop_falling = np.where(np.isnan(image_array/image_array), 0, image_array/image_array)
    
    # strip the falling block from the base array
    future_stub  = drop_falling * unit_img
    # broadcast the stub array to the third dimension to prepare to recieve permutations
    future_stubs = np.broadcast_to(future_stub,(18,*future_stub.shape))
#     print('shape of future_stubs:', future_stubs.shape)

    # find indices of tops of each column
#     print('drop_falling array: \n', drop_falling)
    # col_tops = np.where(drop_falling.sum(axis=0) == 0, drop_falling.shape[0], 
    #                     np.argmax(drop_falling, axis=0)                 )
    col_tops = 13-drop_falling.sum(axis=0)
#     print('col_tops:', col_tops)
    
#     print('falling_perms: \n', falling_perms)
#     print('future_stub: \n', future_stub)

    future_states = future_stubs.copy()

    future_state_scores = np.zeros(18)

    for i in range(18):
        chi = np.zeros(6)
        isi = np.zeros(1)

        row_bottom_target = int(col_tops[i//3])
#         print(row_bottom)
        row_top_target = row_bottom_target-3
#         print(row_top)
        column_target = i//3
#         print(column_target)
        perm_target = i%3
#         print(perm_index)
        try:
            future_states[i, row_top_target:row_bottom_target, 
                          column_target] = falling_perms[:, perm_target]

            cp,ib,chi,isi = col_isle_fit_calc(future_states[i].reshape(future_stub.shape),chi,isi)

            future_state_scores[i] = cp * col_scale + ib * isle_scale

        #if the block won't fit in a column
        except ValueError:
            future_state_scores[i] = -999999999
            
    # if a column gets too high and blocks off those between it and the edge    
    if future_state_scores[3]   == -999999999:
        # give the net a "game over"-level penalty for that state
        future_state_scores[0:3]   = -999999999
    if future_state_scores[6]   == -999999999:
        future_state_scores[0:6]   = -999999999
    if future_state_scores[12] == -999999999:
        future_state_scores[15:]   = -999999999
        
    #     print(f'test_stubs layer {i}: \n', test_stubs[i])
#     print('future state scores:', future_state_scores)
    return future_state_scores

# img = np.zeros((13,6))
# img[5: ,0] = np.random.randint(1,7, size=(8) )
# img[2: ,1] = np.random.randint(1,7, size=(11))
# img[5: ,2] = np.random.randint(1,7, size=(8) )
# img[7: ,3] = np.random.randint(1,7, size=(6) )
# img[0: ,5] = np.random.randint(1,7, size=(13))
# img[0:3,3] = np.random.randint(1,7, size=(3) )

# score_future_states(img,-0.2,30)


In [6]:
def provide_action(nn_choice, seq_index):
    '''
     input: nn_choice (int): selected action from the neural net
            seq_index  (int): v

    output: stepInput (list): multibinary list of button presses for the next frame 
                              of the game

    called by: eval_genomes
    '''

    # actions (length 12)
    #        c1       u d l r    c2
    down  = [0,0,0,0, 0,1,0,0, 0,0,0,0]
    left  = [0,0,0,0, 0,0,1,0, 0,0,0,0]
    right = [0,0,0,0, 0,0,0,1, 0,0,0,0]
    C1    = [1,0,0,0, 0,0,0,0, 0,0,0,0]
    C2    = [0,0,0,0, 0,0,0,0, 0,1,0,0]
    nil   = [0,0,0,0, 0,0,0,0, 0,0,0,0]

    # action sequences (length 15)
    #          0            2           4           6 
    #                 8          10          12          14 
    a_0 = [ left,  left, left, down, down, down, down, 
            down,  down, down, down, down, down,   nil, nil]
    a_1 = [ left,  left, down, down, down, down, down,
            down,  down, down, down, down,  nil,  nil,  nil]
    a_2 = [ left,  down, down, down, down, down, down,
            down,  down, down, down,  nil,  nil,  nil,  nil]
    a_3 = [ down,  down, down, down, down, down, down,
            down,  down, down,  nil,  nil,  nil,  nil,  nil]
    a_4 = [right,  down, down, down, down, down, down,
            down,  down, down, down,  nil,  nil,  nil,  nil]
    a_5 = [right, right, down, down, down, down, down,
            down,  down, down, down, down,  nil,  nil,  nil]

    b_0 = [ left,  left, left,   C1, down, down, down,
            down,  down, down, down, down, down, down,  nil]
    b_1 = [ left,  left,   C1, down, down, down, down,
            down,  down, down, down, down, down,  nil,  nil]
    b_2 = [ left,    C1, down, down, down, down, down,
            down,  down, down, down, down,  nil,  nil,  nil]
    b_3 = [ left,  down, right,  C1, down, down, down, down, down, down,
            down,  down, down, down,  nil,  nil,  nil,  nil]
    b_4 = [right,    C1, down, down, down, down, down,
            down,  down, down, down, down,  nil,  nil,  nil]
    b_5 = [right, right,   C1, down, down, down, down,
            down,  down, down, down, down, down,  nil,  nil]

#     c_0 = [ left,  left, left,   C1, C1, nil, nil, nil, nil, nil, 
#            nil, nil, nil, nil, nil, C2, C2]
    c_0 = [ left,  left, left,   C1,   C1,  nil,  nil,  nil,
             nil,   nil,  nil,  nil,  nil,  nil,  nil,   C2,  C2]
    c_1 = [ left,  left,   C1,   C1,  nil,  nil,  nil,
             nil,   nil,  nil,  nil,  nil,  nil,  nil,   C2,  C2]
    c_2 = [ left,   C1,   C1,  nil,  nil,  nil,
             nil,   nil,  nil,  nil,  nil,  nil,  nil,   C2,  C2]
    c_3 = [ left,   down, right, C1,   C1,   nil,  nil,  nil,
             nil,   nil,  nil,  nil,  nil,  nil,  nil, down, C2]
    c_4 = [right,   C1,   C1,  nil,  nil,  nil,
             nil,   nil,  nil,  nil,  nil,  nil,  nil,   C2,  C2]
    c_5 = [right, right,   C1,   C1,  nil,  nil,  nil,
             nil,   nil,  nil,  nil,  nil,  nil,  nil,   C2,  C2]
    # sequence options (length 18)                 
    choices = [a_0, a_1, a_2, a_3, a_4, a_5, 
               b_0, b_1, b_2, b_3, b_4, b_5, 
               c_0, c_1, c_2, c_3, c_4, c_5 ]
    # if the inputs are not in actionable ranges
    if (nn_choice not in range(0,len(choices))) or (seq_index not in range(0,len(c_0)+1)):
        # take no action
        return down
    # otherwise set stepInput appropriately
    if seq_index%4== 0: 
        stepInput = choices[nn_choice][int(seq_index//4)]
    else:
        stepInput = nil
    return stepInput

In [7]:
def eval_genomes(genomes, config, slowdown='', render=True, cv2_disp=True):
    '''
     input: genomes (neat.genome.DefaultGenome): genome objects from neat.population.
                                                 Population to be evaluated
            config (neat.config.Config): neat configuration container
            slowdown (float or int): slow down each frame of game advancement by 
                                     the number of seconds specified
            render (bool): indicate whether to render the game via gym-retro
            cv2_disp (bool): indicate whether to display neural net input via cv2
           
    output: [N/A or fitness(float)]
    
    called by: neat.population.Population.run
    '''
    # initialize genome counter and action frequency
    genome_counter = 0
    # define shrink factor and pre-shrink cropped image dimensions
    shrink, dims = 8, np.array([8, 208, 16, 96])
    # define size of processed game image after crop and shrink
    sc_dims = dims/shrink
    # define reward drought counter limit
    seq_index = 0
    # define reward scaling factors
    col_scale, isle_scale, match_scale, wild_scale = -0.2, 30, 500, 5
    rescale_gray_map = rescale_grays()

    # reset the environment to initialize gym retro
    ob = env.reset()    
    
    # for each member of the population:
    for genome_id, genome in genomes:

        # increment genome counter
        genome_counter += 1
      
        # initialize a few tracking variables
        fitness, d_fitness, seq_counter, nn_choice, frame = 0, 0, -3, 3, 0
        # column height and island size starting states to feed col_isle_fit_calc
        cols_i, isles_i = np.zeros(6), np.array(0)
        # initialize being done as false
        done = False
        
        # create the neural net
        net = neat.nn.recurrent.RecurrentNetwork.create(genome, config)

        # reset the environment
        ob = env.reset()
        
        # if viewing the neural net input directly is desired
        if cv2_disp == True:
            # make an OpenCV window
            cv2.namedWindow('Decisive play', cv2.WINDOW_NORMAL)
        
        # begin tracking of cycle time
        start_time = time.time()
        
        # while done is not True, i.e., the game is running
        while not done:           
            # open a render window to follow the action if requested
            if render == True:
                env.render()
            # delay each frame for slowdown seconds if requested
            if slowdown != '':
                time.sleep(slowdown)
            # increment the frame counter
            frame += 1
#             print('Frame', frame)
            # process the image to feed the neural net
            resc_img = process_image(ob, rescale_gray_map)
            # show the cv2 window to follow the game state if requested 
            if cv2_disp == True:
                cv2.imshow('Decisive play', resc_img)
                cv2.waitKey(1)   


            # evaluate column height penalty and color island bonus 
            col_pen, isle_bonus, cols_i, isles_i = col_isle_fit_calc(resc_img, cols_i, isles_i)

            # build the input to the neural net
            img_array = np.ravel(resc_img)
            
            if np.all(resc_img[0:3,3]) != 0: #and seq_counter == 0:
#                 print(np.all(resc_img[0:3,3]) != 0 , "seq_counter", seq_counter)
                if seq_counter == 0:
                    # pre-score the future states when the current block is placed
                    future_scores = score_future_states(resc_img, col_scale, isle_scale)

                    # concatenate future scores with the game board state at evaluation
                    nnInput = np.append(d_fitness, np.append(future_scores, img_array))

                    # let neural net do its thing on input and generate action 
                    nnOutput = np.array(net.activate(nnInput))
#                     print('nnOutput:', nnOutput)
                    nn_choice = np.random.choice(np.ravel(np.indices(nnOutput.shape).reshape(nnOutput.shape)
                                                 *[nnOutput==np.max(nnOutput)]))
#                     nn_choice = 11
                    # apply column penalty to fitness
                    d_fitness += float(col_pen) * col_scale
                    # apply scaled isle creation bonus to fitness
                    d_fitness += float(isle_bonus) * isle_scale
#                     print([nnOutput==np.max(nnOutput)])

#                     print('choice',  nn_choice)
                        #
            stepInput = provide_action(nn_choice, seq_counter)
            # if frame = 1 or 2...
            if frame <= 3:
                # .. press down once (total twice) to fully expose the falling block
                stepInput = [0, 0, 0, 0,  0, 1, 0, 0,  0, 0, 0, 0]

#             print('stepInput', stepInput, 'seq_counter', seq_counter )
            seq_counter += 1
#                         print('Frame', frame, 'NN output:', nnOutput)
#                         print('nn', time.time() - start_time, 's'); start_time = time.time()
                
        #             print('env step:', time.time() - start_time, 's'); start_time = time.time()
            # feed the specified action to the emulator and capture outputs
#             print('Frame', frame, 'action:', step_input)
            ob, rew, done, info = env.step(stepInput)
            # store score separately for reporting because f-strings are persnickety
            step_score = info['score'] 
            # if a new block is fully visible
            if np.all(resc_img[0:3,3]) != 0 and seq_counter > 3:
                #reset sequence counter
                seq_counter = 0
            # if a reward is earned, i.e., score goes up
            if (rew > 1):
                # when removing gems:
                if (rew != 10000):
                    # apply scaled reward to fitness 
                    d_fitness += rew * match_scale
                # when wildcard/magic block hits bottom:
                elif rew == 10000:
                    # apply scaled reward to fitness
                    d_fitness += rew * wild_scale
            # when pressing down to simply speed play:
            else:    
                # add the minor 'down' bonus to the fitness when it occurs
                d_fitness += rew    


            # add the fitness differential to the overall fitness
            fitness += d_fitness
            
            # when a significant reward is earned...
            if rew >= 30:
                # ... give status update
                print(f"Frame {frame}: Earned {rew} reward.  Score: {step_score}  \
                      Fitness change: {d_fitness:0.3f}")

        # once out of the while loop, i.e., when the run is done
        # ... print out the high-level results...
        print(f'ID:{genome_id} S/N: {genome_counter} Final frame: {frame} \
              Fitness: {fitness:0.3f} Score: {step_score}')
        # ... and the game over indicator
        print(f'                             ****** }}}}>>>>>ID \
                  {genome_id} GAME OVER after {(time.time() - start_time):.3f} \
                  s <<<<<{{{{ ******'                                           )
        
        # save final fitness to genome_id's fitness attribute 
        genome.fitness = fitness
        # return fitness to the parallel.ParallelEvaluator (Population.run ignores)
#         return fitness


In [8]:
# close the render window when finished
try:
    env.render(close=True)
    env.close()
except:
    pass


In [ ]:
######## Major parameter specifications ########

# specify game to load with its config file (comment/uncomment as appropriate)
game, state, config_file = 'Columns-Genesis', 'Arcade.Easy.Level0', 'config-Columns-Decisive'

# specify checkpoint file if necessary as a string ('0' if not to be used)
chkpt = '0'
# specify checkpoint index manually
# chkpt_index = 1
# chkpt = game + '-neat-chkpt-dec-'+ str(chkpt_index)

# specify type of retro environment observation space via index of below list
input_type_index = 0 
# specify observation type
input_types = [retro.Observations.IMAGE, retro.Observations.RAM]

######## END Major parameter specifications ########

# create the retro environment for the chosen game, including starting state
env = retro.make(game,state,obs_type=input_types[input_type_index],
                 use_restricted_actions=retro.Actions.ALL          )

# restore the checkpoint if specified
if chkpt != '0':
    print(f"Opening checkpoint file {chkpt}...")
    p = neat.Checkpointer.restore_checkpoint(chkpt)

# load in the configuration for the NEAT algorithm...
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction, 
                     neat.DefaultSpeciesSet, neat.DefaultStagnation, config_file)
    
# ... and create the NEAT population with the above specified configuration
pop = neat.Population(config)

# prepare statistics reporting    
pop.add_reporter(neat.StdOutReporter(True))

stats = neat.StatisticsReporter()

pop.add_reporter(stats)

# and create checkpoint file as things progress
pop.add_reporter(neat.Checkpointer(generation_interval = 1, 
                                 time_interval_seconds = 300,
                                 filename_prefix = game + '-neat-chkpt-dec-'))

# set about the evolution process
winner = pop.run(eval_genomes)

# pickle the final result
with open(game+'-winner.pkl', 'wb') as output:
    pickle.dump(winner, output, 1)

# close the render window when finished
try:
    env.render(close=True)
    env.close()
except:
    pass


Opening checkpoint file Columns-Genesis-neat-chkpt-dec-1...

 ****** Running generation 0 ****** 

Frame 634: Earned 60.0 reward.  Score: 216                        Fitness change: 30156.000
Frame 894: Earned 30.0 reward.  Score: 300                        Fitness change: 45210.000
Frame 947: Earned 120.0 reward.  Score: 420                        Fitness change: 105210.000
Frame 1227: Earned 30.0 reward.  Score: 520                        Fitness change: 120280.000
ID:1 S/N: 1 Final frame: 1538               Fitness: 77303578.000 Score: 575
                             ****** }}>>>>>ID                   1 GAME OVER after 42.819                   s <<<<<{{ ******
Frame 472: Earned 30.0 reward.  Score: 155                        Fitness change: 15125.000
Frame 1058: Earned 180.0 reward.  Score: 466                        Fitness change: 105256.000
Frame 1236: Earned 30.0 reward.  Score: 516                        Fitness change: 120276.000
Frame 1566: Earned 30.0 reward.  Score: 614    

Frame 2003: Earned 240.0 reward.  Score: 11096                        Fitness change: 410376.000
Frame 2675: Earned 30.0 reward.  Score: 11227                        Fitness change: 425477.000
Frame 2859: Earned 60.0 reward.  Score: 11313                        Fitness change: 468477.000
ID:10 S/N: 10 Final frame: 2884               Fitness: 472737110.000 Score: 11313
                             ****** }}>>>>>ID                   10 GAME OVER after 65.672                   s <<<<<{{ ******
Frame 773: Earned 10000.0 reward.  Score: 10192                        Fitness change: 50192.000
Frame 1167: Earned 30.0 reward.  Score: 10306                        Fitness change: 65276.000
Frame 1220: Earned 60.0 reward.  Score: 10366                        Fitness change: 95276.000
Frame 1273: Earned 90.0 reward.  Score: 10456                        Fitness change: 140276.000
Frame 1440: Earned 30.0 reward.  Score: 10498                        Fitness change: 155288.000
Frame 1622: Earned 30.0 r

Frame 777: Earned 10000.0 reward.  Score: 10192                        Fitness change: 50192.000
Frame 1438: Earned 30.0 reward.  Score: 10366                        Fitness change: 65336.000
ID:24 S/N: 24 Final frame: 1573               Fitness: 42200606.000 Score: 10377
                             ****** }}>>>>>ID                   24 GAME OVER after 34.701                   s <<<<<{{ ******
Frame 616: Earned 30.0 reward.  Score: 197                        Fitness change: 15167.000
Frame 860: Earned 30.0 reward.  Score: 263                        Fitness change: 30203.000
Frame 1036: Earned 180.0 reward.  Score: 478                        Fitness change: 120238.000
Frame 1241: Earned 30.0 reward.  Score: 550                        Fitness change: 135280.000
Frame 1558: Earned 30.0 reward.  Score: 636                        Fitness change: 150336.000
Frame 1752: Earned 30.0 reward.  Score: 702                        Fitness change: 165372.000
ID:25 S/N: 25 Final frame: 2348          